# Agent Tools
- https://python.langchain.com/docs/tutorials/agents/ 
- https://python.langchain.com/docs/integrations/tools/

In [ ]:
# llm模型設定
# https://build.nvidia.com/deepseek-ai/deepseek-r1
# nvapi-
import getpass
import os
if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

In [ ]:
!uv pip install -qU "langchain-nvidia-ai-endpoints"

In [ ]:
## 模型來源
# https://build.nvidia.com/deepseek-ai
# https://build.nvidia.com/meta
# https://build.nvidia.com/google
# https://build.nvidia.com/qwen

from langchain.chat_models import init_chat_model
llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")

In [ ]:
# 工具列表
from langchain.agents import get_all_tool_names
get_all_tool_names()

In [ ]:
# llm模型設定
# https://openweathermap.org/
# b57
import getpass
import os
if not os.environ.get("OPENWEATHERMAP_API_KEY"):
  os.environ["OPENWEATHERMAP_API_KEY"] = getpass.getpass("Enter API key for OPENWEATHERMAP_API_KEY: ")

In [ ]:
# llm模型設定
# https://tavily.com/
# tvly-
import getpass
import os
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for TAVILY_API_KEY: ")

In [ ]:
!uv pip install -qU "pyowm"

In [ ]:
import os
from langchain.agents import AgentType, initialize_agent, load_tools
tools = load_tools(["openweathermap-api"], llm)
agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_chain.run("What's the weather like in London?")

In [ ]:
# 外部tools
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
tools = [search]
agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_chain.run("今天台灣新聞 白沙屯")

In [ ]:
!uv pip install -U duckduckgo-search

In [ ]:
# 混合 tools
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
tool_names_to_load = ["ddg-search"]
loaded_named_tools = load_tools(tool_names_to_load, llm=llm)
tools = loaded_named_tools + [search]
agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_chain.run("今天台灣新聞 關於白沙屯, 及台中天氣")

## 自建tools

In [ ]:
from langchain.tools import tool

@tool
def get_weather(city: str) -> dict:
    """Get weather for a given city"""
    print(f"[TOOL] get_weather 被呼叫，城市：{city}")    
    return {
        "location": city,
        "temperature": "23°C",
        "condition": "Sunny ☀️",
        "播報員": "mary"
    }



In [ ]:
!uv pip install pytz

In [ ]:
from langchain.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone: str) -> str:
    """根據時區取得目前的日期與時間。"""
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.now(tz)
        return current_time.strftime('%Y-%m-%d %H:%M:%S')
    except pytz.UnknownTimeZoneError:
        return f"無法識別的時區: {timezone}"


In [ ]:
#from tools_time import get_current_time  # 同時引入這兩個工具
#from tools_openweather import get_openweather  # 同時引入這兩個工具

tool_names_to_load = ["ddg-search"]
loaded_named_tools = load_tools(tool_names_to_load, llm=llm)
tools = loaded_named_tools + [get_weather, get_current_time]
agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_chain.run("今天台中天氣及現在時間及白沙屯進度")

## 作業
1.  **Agent 建立與 API 整合：**
    * 使用 Langchain 建立一個 Agent，並整合至少一個外部 API。
    * 建議可考慮的 API 包括：
        * 台灣高鐵 Open API：查詢時刻表、票價、剩餘座位等。
        * 中央氣象局 Open API：查詢天氣預報、觀測資料等。
        * Google Maps Platform API：查詢地點資訊、導航路線等。
        * 金融監督管理委員會 Open API：查詢股票資訊、匯率資訊等。
    * 你也可以選擇其他你感興趣的 API，但請在報告中說明其功能和應用情境。

2.  **Agent 行為設計：**
    * Agent 應分析使用者意圖，判斷是否需要呼叫外部 API 才能回答問題。
    * 如果使用者詢問的資訊可以透過 API 取得，則 Agent 必須使用 API。
    * 如果無法透過 API 取得，則 Agent 應嘗試使用 LLM 回答。
    * Agent 應向使用者說明其使用的工具（API 或 LLM）以及原因。

3.  **範例互動：**
    * **範例輸入 1：**「請問明天早上從台北到左營的高鐵有哪些車次？」
    * **範例輸出 1：**「我查詢了台灣高鐵 API，明天早上從台北到左營的高鐵車次有以下幾班：...」
    * **範例輸入 2：**「台北 101 有多高？」
    * **範例輸出 2：**「台北 101 的高度是 508 公尺。」（此資訊可能透過 LLM 或 Google Maps API 取得）

4.  **進階功能（選做）：**
    * 整合多個 API，回答更複雜的問題。
    * 加入錯誤處理機制，提升 Agent 的健壯性。
    * 讓 Agent 能夠處理需要多輪對話才能完成的任務。
    * 設計一個簡單的使用者介面，讓使用者可以更方便地與 Agent 互動。

**評估標準：**

* Agent 是否能夠成功整合外部 API？ (40%)
* Agent 是否能夠根據使用者意圖正確判斷是否使用 API？ (40%)
* Agent 的回答是否準確、完整、符合使用者需求？ (20%)

